In [1]:
import requests
from bs4 import BeautifulSoup
import json
# ------- STAGE 1 -------
# Interaction Code
url = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=15174;type=tournament'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [2]:
# Parser Code
links = []
table_rows = soup.select('table.engineTable > tbody > tr.data1')
for row in table_rows:
    match_url = "https://www.espncricinfo.com" + row.select('td')[6].select('a')[0]['href']
    links.append(match_url)
print(links)
# Next Stag

['https://www.espncricinfo.com/ci/engine/match/1358929.html', 'https://www.espncricinfo.com/ci/engine/match/1358930.html', 'https://www.espncricinfo.com/ci/engine/match/1358931.html', 'https://www.espncricinfo.com/ci/engine/match/1358932.html', 'https://www.espncricinfo.com/ci/engine/match/1358933.html', 'https://www.espncricinfo.com/ci/engine/match/1358934.html', 'https://www.espncricinfo.com/ci/engine/match/1358935.html', 'https://www.espncricinfo.com/ci/engine/match/1358936.html', 'https://www.espncricinfo.com/ci/engine/match/1358937.html', 'https://www.espncricinfo.com/ci/engine/match/1358938.html', 'https://www.espncricinfo.com/ci/engine/match/1358939.html', 'https://www.espncricinfo.com/ci/engine/match/1358940.html', 'https://www.espncricinfo.com/ci/engine/match/1358941.html', 'https://www.espncricinfo.com/ci/engine/match/1358942.html', 'https://www.espncricinfo.com/ci/engine/match/1358943.html', 'https://www.espncricinfo.com/ci/engine/match/1358944.html', 'https://www.espncricin

In [3]:
bowling_summary = []

In [4]:
for url in links:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    match = soup.find('span', text='MATCH FLOW').parent.parent.next_sibling.find_all_next('div')
    team1 = match[1].select('span')[0].text.replace("innings","")
    team2 = match[0].select('span')[0].text.replace("innings","")
    match_info = team1 + ' Vs ' + team2   
    tables = soup.select('div > table.ds-table')
    first_inning_rows = [row for row in tables[1].select('tbody > tr') if len(row.select('td')) >= 11]
    second_inning_rows = [row for row in tables[3].select('tbody > tr') if len(row.select('td')) >=11]
    summary = []
    try:
        
       for i in range(0,len(first_inning_rows)+1):
            row = first_inning_rows[i]

            cols = row.select('td')
            summary.append({
                'match': match_info,
                'bowlingTeam': team1,
                'bowlerName': cols[0].select('a > span')[0].text,
                'overs': cols[1].text,
                'maiden': cols[2].text,
                'runs': cols[3].text,
                'wickets': cols[4].text,
                'economy': cols[5].text,
                '0s': cols[6].text,
                '4s': cols[7].text,
                '6s': cols[8].text,
                'wides': cols[9].text,
                'noBalls': cols[10].text
            })
            print(summary)
           
           
    except Exception as e:
        print(e)
    
    try:        
        for i in range(0,len(second_inning_rows)+1):
            row = second_inning_rows[i]
            cols = row.select('td')
            summary.append({
                'match': match_info,
                'bowlingTeam': team2,
                'bowlerName': cols[0].select('a > span')[0].text,
                'overs': cols[1].text,
                'maiden': cols[2].text,
                'runs': cols[3].text,
                'wickets': cols[4].text,
                'economy': cols[5].text,
                '0s': cols[6].text,
                '4s': cols[7].text,
                '6s': cols[8].text,
                'wides': cols[9].text,
                'noBalls': cols[10].text
            })
       
    except Exception as e:
        print(e)
    bowling_summary.append(summary)

[{'match': 'Gujarat Giants Women  Vs Mumbai Indians Women ', 'bowlingTeam': 'Gujarat Giants Women ', 'bowlerName': 'Ashleigh Gardner', 'overs': '4', 'maiden': '0', 'runs': '38', 'wickets': '1', 'economy': '9.50', '0s': '7', '4s': '7', '6s': '0', 'wides': '0', 'noBalls': '0'}]
[{'match': 'Gujarat Giants Women  Vs Mumbai Indians Women ', 'bowlingTeam': 'Gujarat Giants Women ', 'bowlerName': 'Ashleigh Gardner', 'overs': '4', 'maiden': '0', 'runs': '38', 'wickets': '1', 'economy': '9.50', '0s': '7', '4s': '7', '6s': '0', 'wides': '0', 'noBalls': '0'}, {'match': 'Gujarat Giants Women  Vs Mumbai Indians Women ', 'bowlingTeam': 'Gujarat Giants Women ', 'bowlerName': 'Mansi Joshi', 'overs': '2', 'maiden': '0', 'runs': '17', 'wickets': '0', 'economy': '8.50', '0s': '7', '4s': '2', '6s': '1', 'wides': '1', 'noBalls': '0'}]
[{'match': 'Gujarat Giants Women  Vs Mumbai Indians Women ', 'bowlingTeam': 'Gujarat Giants Women ', 'bowlerName': 'Ashleigh Gardner', 'overs': '4', 'maiden': '0', 'runs': '38

In [5]:
print(bowling_summary)

[[{'match': 'Gujarat Giants Women  Vs Mumbai Indians Women ', 'bowlingTeam': 'Gujarat Giants Women ', 'bowlerName': 'Ashleigh Gardner', 'overs': '4', 'maiden': '0', 'runs': '38', 'wickets': '1', 'economy': '9.50', '0s': '7', '4s': '7', '6s': '0', 'wides': '0', 'noBalls': '0'}, {'match': 'Gujarat Giants Women  Vs Mumbai Indians Women ', 'bowlingTeam': 'Gujarat Giants Women ', 'bowlerName': 'Mansi Joshi', 'overs': '2', 'maiden': '0', 'runs': '17', 'wickets': '0', 'economy': '8.50', '0s': '7', '4s': '2', '6s': '1', 'wides': '1', 'noBalls': '0'}, {'match': 'Gujarat Giants Women  Vs Mumbai Indians Women ', 'bowlingTeam': 'Gujarat Giants Women ', 'bowlerName': 'Tanuja Kanwar', 'overs': '2', 'maiden': '0', 'runs': '12', 'wickets': '1', 'economy': '6.00', '0s': '6', '4s': '2', '6s': '0', 'wides': '0', 'noBalls': '0'}, {'match': 'Gujarat Giants Women  Vs Mumbai Indians Women ', 'bowlingTeam': 'Gujarat Giants Women ', 'bowlerName': 'Monica Patel', 'overs': '2', 'maiden': '0', 'runs': '34', 'wick

In [6]:
with open('bowling_summary.json', 'w') as f:
    json.dump(bowling_summary, f, indent=4)